### Les selvsamlede data inn i python

Dette skriptet er et eksempel på hvordan dere kan lese deres egne data inn i python. Datasettet her er hentet inn av Ragnhild Hannaas med samme type logger som det dere bruker, så det skal se nokså likt ut for dere. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Dette er stien til filen du vil laste inn. Bytt den ut med stien til der du har de selvinnhentede dataene dine. 

In [ ]:
sti='W:/Work/Documents/EkteData/EkteData/data'
fil='/4-6_januar_Temperaturegistrering_AB-vei.csv'

Forklaring på keywordsene i filinnlesingen under: 
- `sep=','`: kolonnene i csv-filen er skilt med komma. Dette er default for csv-filer, så denne spesifiseringen er strengt tatt overflødig. 
- `header=0`: den første linjen i filen er en "header" altså ikke faktiske data men forklaring på dataene. 
- `engine=python`: siden Celcius en angitt med grad-tegn (special character) må man spesifisere engine. 
- `parse:dates=['Time']`: konverterer kolonnen med tid til "Timestamp" som er lett å jobbe med for tid. 

In [ ]:
df=pd.read_csv(
    sti+fil, sep=',', header=0, engine='python',
    parse_dates=['Time']
)

In [ ]:
df

Konverter pandas DataFramen til et numpy array. Dette tar hånd om tidsaksen bedre når man plotter. 

In [ ]:
data=df.to_numpy()

In [ ]:
data

Lagre tidskolonnen og temperaturkolonnen i hver sine vektorer: `tid` og `temp`

In [ ]:
tid=data[:,1]
temp=data[:,2]

Lag en figur for å inspisere datasettet. 

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(tid,temp)
plt.xlim(tid[0], tid[-1])

Dette ser litt rart ut - her trenger vi å vite litt om datainnhentingsprosessen for å forstå hva som må forklares av feilkilder og hva som er faktiske temperaturendringer. 

Det er for eksempel to hopp i temperatur: ett etter kl 12 den 5. januar, og ett helt på slutten av tidsserien. Er instrumentet blitt flyttet? Lå instrumentet i skyggen eller i solen? Når man samler inn data er det viktig å holde oversikten over slike forhold som kan påvirke datasettet. 

En annen ting som kan være lurt å sjekke om man bruker et instrument man ikke kjenner til så godt er om den har lagret data med konstant frekvens. I figuren over ser det litt ut som om at noen perioder har høyere oppløsning enn andre. Er det hull i datasettet eller er bare temperaturen veldig jevn i noen perioder?

In [ ]:
pd.infer_freq(df['Time'])
